# SRS : MagLPI for COMET experiment

Objectif : [...]

How to proceed [...]


In [1]:
# **********************************
# First Run This Cell to 
# import the necessary libraries
# **********************************
#
# Please run this cell before running anything, and run this cell again if you have restarted the 
# python kernel.
#
# This cell imports useful routines to diagnose the simulations here
#

import IPython.display
import sys
import subprocess
sys.path.append("../../analysis/")
%matplotlib inline
import osiris
from osiris import tajima
from srs_helper import *
import numpy as np
import matplotlib.pyplot as plt
from osh5vis import osplot
from osh5io import *
from osh5utils import fft
from ipywidgets import interact_manual,Layout,interact, FloatSlider
import ipywidgets as widgets
interact_calc=interact_manual.options(manual_name="Make New Input and Run")

/!\ Si le champ E n'est pas compilé en un seul fichier hdf5. =>> RUN la cell suivante

In [2]:
def execute(cmd):

    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, universal_newlines=True)
    for stdout_line in iter(popen.stdout.readline, ""):
        yield stdout_line
    popen.stdout.close()
    return_code = popen.wait()
    if return_code:
        raise subprocess.CalledProcessError(return_code, cmd)

def combine_h5_1d(ex ,rundir, print_out = 'yes'):
    in_file = rundir + '/MS/FLD/' + ex + '/'
    out_file = rundir + '/' + ex + '.h5'
    for path in execute(["python3", "/home/sbolanos/Documents/code/JupyterPIC/analysis/combine_h5_util_1d.py", in_file, out_file]):
        if print_out == 'yes':
            IPython.display.clear_output(wait=True)
#            print(path, end='')

rundir = './winjum/SRS_Bx10_gradn_Te8e-2_upS'

print('combining E1 files')
combine_h5_1d('e1',rundir)
combine_h5_1d('b1',rundir)
print('DONE')
print('combining E2 files')
combine_h5_1d('e2',rundir)
combine_h5_1d('b2',rundir)
print('DONE')
print('combining E3 files')
combine_h5_1d('e3',rundir)
combine_h5_1d('b3',rundir)
print('DONE')

CalledProcessError: Command '['python3', '/home/sbolanos/Documents/code/JupyterPIC/analysis/combine_h5_util_1d.py', './winjum/SRS_Bx10_gradn_Te8e-2_upS/MS/FLD/e1/', './winjum/SRS_Bx10_gradn_Te8e-2_upS/e1.h5']' returned non-zero exit status 1.

# ANALYSE DATA

In [ ]:
dirname = './winjum/SRS_B_gradn_Te8e-2_upS'

osiris.plot_tk_arb(dirname,'e1',klim=0.4,tlim=6000)

osiris.plot_tk_arb(dirname,'e2',klim=0.4,tlim=6000)

In [ ]:
osiris.plot_xt_arb(rundir=dirname,field='e1',vmax=0.0005,vmin=-0.0005,cmap='bwr')

osiris.plot_xt_arb(rundir=dirname,field='e2',vmax=0.0005,vmin=-0.0005,cmap='bwr',)

In [ ]:
path_data = dirname + '/e2.h5'
e2 = osh5io.read_h5(path_data)

plt.imshow(e2[4000:5000,1000:2000],origin='lower')
plt.colorbar()

In [ ]:
plt.plot(e2[1000,300:401])
plt.grid()

In [ ]:
srs_movie(dirname)

In [ ]:
osiris.plot_wk(rundir=dirname, wlim=[0,0.02], klim=[0,16], vth = 0.08, vmin=-1, vmax=3)

# External B-field


In [ ]:
dirname = './winjum/SRS_B_Te1e-3_ext'

osiris.plot_xt_arb(rundir=dirname,field='ext_b1',tlim=(0,3000),xlim=(0,2148),vmax=0.003,vmin=-0.001)

osiris.plot_xt_arb(rundir=dirname,field='ext_b2',tlim=(0,3000),xlim=(0,2148))

# SANDBOX

In [ ]:
from scipy.optimize import curve_fit

dirname =  './winjum/SRS_B_gradn_Te8e-2_upS'

def poynting(x,S,w,pha):
    return S*(np.sin(w*x+pha))**2

s1,s2,s3 = osiris.get_Poynting(dirname)

ne_cr=0.13

w =1.#00001#np.sqrt(1.+ne_cr)
S0= 4.4e-4

nt = len(s1.axes[0].get_axis_points())
t = s1.axes[0].get_axis_points()
#plt.imshow(s1[:,0:100],origin="lower")
#plt.colorbar()

S_ref = S0*(np.sin(w*t))**2

plt.plot(t,s1.data[:,1],label='OSIRIS')
plt.plot(t,S_ref,label='REF')
plt.plot(t,S_ref-s1.data[:,1])
plt.legend(loc='best')
plt.grid()


In [ ]:
plt.plot(t,s1.data[:,1],label='OSIRIS')
plt.plot(t,S_ref,label='REF')
plt.plot(t,S_ref-s1.data[:,1])
plt.legend(loc='best')
plt.grid()
plt.xlim([4400.,4500.])

In [ ]:
plt.plot(t,s1.data[:,1],label='OSIRIS')
plt.plot(t,S_ref,label='REF')
plt.plot(t,S_ref-s1.data[:,1])
plt.legend(loc='best')
plt.grid()
plt.xlim([t[imin],t[imax]])

In [ ]:
imin = 100
imax = 1500
popt, pcov = curve_fit(poynting, t[imin:imax], s1.data[imin:imax,1])

print('results: ',popt)

In [ ]:

S_ref = poynting(t,*popt)

plt.plot(t,s1.data[:,1],label='OSIRIS')
#plt.plot(t,S_ref,label='REF')
plt.plot(t,S_ref-s1.data[:,1])
plt.legend(loc='best')
plt.grid()

In [ ]:
i_init = 0#1500

int_s1 = np.sum(s1.data[i_init:,1])
int_ref= np.sum(S_ref[i_init:])

reflect= (S_ref-s1.data[:,1])
sum_ref=np.sum(reflect)/int_ref

print('OSIRIS poynting:',int_s1)
print('REFER  poynting:',int_ref)
print('% Reflectivity :',(int_ref-int_s1)/int_ref)


# Roman Lee: Reflectivity measurement

In [ ]:
s1.data.shape

In [ ]:
data_h5 = s1
n_savg=200 # cell in space around which to take spatial average, this smooths out reflectivity data
nn_avg=100  # nbr of cell to average arounde n_savg
c = s1.data

# average in time around nth spatial cell
tmp = np.mean( c[:, n_savg-nn_avg:n_savg+nn_avg ], axis = 1 )

# TODO this is a total hack, need a more reliable way to get amplitude of laser
# Although, it's easy to tell by looking at the plot if it didn't work
rmin =400 
rmax = int(len(tmp)/7)
print('rmax =',rmax)
cmax = np.max(tmp[rmin:rmax]) # should be poynting flux when only laser is present at t_rise, no scattering
print('Cmax = ', cmax)

# average reflectivity
ave_ref = (cmax - np.average(c))/cmax 
print('% ave Reflectivity: ',ave_ref)


rr = (cmax - tmp)/cmax

plt.plot(t,c[:,n_savg])
plt.plot(t,tmp)
#save='reflectivity'
#suptitle='suptitle'
#subplot_title='subplot_title'
#roman = plot_reflectivity(data_h5,n_savg,save,suptitle,subplot_title,afs=(8,8))

In [ ]:
plt.plot(t,reflect/popt[0])
plt.plot(t,rr)
#plt.xlim([300,1700])
#plt.ylim([-0.01,0.02])

In [ ]:
tt = np.mean(c[ :, 1:2*n_savg ],axis=1)
tt.shape

In [ ]:
plt.plot(t,tmp)